In [1]:
# Initial setup

# Import of relevant packages
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from sklearn import metrics
from timeit import default_timer as timer
from sklearn.naive_bayes import CategoricalNB

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import roc_curve, confusion_matrix, accuracy_score, recall_score, precision_score

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
# Set random seed 
RSEED = 42
warnings.filterwarnings("ignore")

#### This data consists prices for SELLING per each 'sq.m' for the apartments of sizes 30m2, 60m2 , 100m2

In [2]:
#Extracting the data from the website

df = pd.read_html('https://www.wohnungsboerse.net/immobilienpreise-Berlin/2825')
df1 = df[1]
df1

### Extracting the data that we need for analysing the prices different Sq.m for Germany

In [25]:
# Extracting the columns and reversing the order using (iloc[::-1])
df2 = df1.loc[1:11, 'DE'].iloc[::-1].reset_index()
df3 = df1.loc[13:23,'DE'].iloc[::-1].reset_index()
df4 = df1.loc[25:35,'DE'].iloc[::-1].reset_index()

# joining all the new columns we want
df_W = pd.concat([df2,df3,df4],axis=1)

# Adding the years column
df_W.loc[:,'year'] = [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021] 
df_W.drop(columns='index',inplace=True,axis=1)

# Assigning the column names
df_W.columns = ['30m2','60m2','100m2','year']

# Ordering the columns to have year at the beginning
df_W = df_W[['year','30m2','60m2','100m2']]

df_W

,year,30m2,60m2,100m2
0,2011,"1.411,03 €","1.520,60 €","2.168,36 €"
1,2012,"1.690,04 €","1.596,73 €","2.261,62 €"
2,2013,"1.929,90 €","1.789,22 €","2.499,54 €"
3,2014,"2.233,51 €","2.044,30 €","2.754,88 €"
4,2015,"2.480,27 €","2.359,35 €","2.950,97 €"
5,2016,"2.552,38 €","2.396,09 €","3.022,58 €"
6,2017,"2.300,03 €","2.264,10 €","2.841,54 €"
7,2018,"3.052,63 €","2.825,95 €","3.405,71 €"
8,2019,"3.600,52 €","3.010,40 €","3.593,52 €"
9,2020,"4.021,14 €","3.466,08 €","3.916,01 €"


#### This data consists prices for RENT for each 'sq.m' for the apartments of sizes 30m2, 60m2 , 100m2

In [ ]:
#Extracting the data from the website

dfr = pd.read_html('https://www.wohnungsboerse.net/mietspiegel-Berlin/2825')
dfr = dfr[1]
dfr

### Extracting the data that we need for analysing the rents different Sq.m for Germany

In [29]:
# Extracting the columns and reversing the order using (iloc[::-1])
df_2 = dfr.loc[1:11, 'DE'].iloc[::-1].reset_index()
df_3 = dfr.loc[13:23,'DE'].iloc[::-1].reset_index()
df_4 = dfr.loc[25:35,'DE'].iloc[::-1].reset_index()

# joining all the new columns we want
df_R = pd.concat([df_2,df_3,df_4],axis=1)

# Adding the years column
df_R.loc[:,'year'] = [2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021] 
df_R.drop(columns='index',inplace=True,axis=1)

# Assigning the column names
df_R.columns = ['30m2','60m2','100m2','year']

# Ordering the columns to have year at the beginning
df_R = df_R[['year','30m2','60m2','100m2']]

df_R

,year,30m2,60m2,100m2
0,2011,"7,63 €","5,56 €","6,11 €"
1,2012,"7,91 €","5,69 €","6,31 €"
2,2013,"8,38 €","5,93 €","6,72 €"
3,2014,"8,62 €","6,12 €","7,02 €"
4,2015,"9,41 €","6,56 €","7,61 €"
5,2016,"11,59 €","7,34 €","8,41 €"
6,2017,"9,68 €","7,37 €","7,49 €"
7,2018,"11,06 €","7,59 €","8,88 €"
8,2019,"11,33 €","7,83 €","9,13 €"
9,2020,"11,41 €","7,91 €","9,15 €"
